In [ ]:
from pathlib import Path
import datetime
#
from dotted_dict import DottedDict
import torch
import torch.nn as nn
#
import numpy as np
import pprint
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
#
import torch.nn.functional as F
import matplotlib.pyplot as plt
#
from torch.utils.tensorboard import SummaryWriter

In [ ]:
from models.backbones import *
from models.projectors import *
from models.barlow_twins import BarlowTwins
from optimizers import *
from augmentations import SimSiamAugmentation, Augmentation
from datasets import get_dataset
from utils import show, show_batch, save_checkpoint
from config_utils import get_dataloaders_from_config, get_config_template, add_paths_to_confg
from train_utils import down_knn, down_train_linear, down_valid_linear, std_cov_valid

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
config = get_config_template()

#################
# DVICE
#################
config.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

#################
# frequencies
#################
config.freqs = {
    "ckpt": 20,
    "lin_eval": 5,
    "knn_eval": 5,
    "std_eval": 5,
    "plot_rec": 20
}
#################
# data
#################
config.p_data = "/mnt/data/pytorch"
config.dataset = "cifar10"
config.img_size = 64
config.n_classes = 10
config.train_split = 'train'
config.down_train_split = 'train'
config.down_valid_split = "valid"
config.augmentations_train = [
    ("RandomResizedCrop", {'size': config.img_size, "scale": (0.2, 1.0)}),
    ("RandomHorizontalFlip", {'p': 0.5}),
    ("RandomApply", {
        "transforms": [
            ("ColorJitter", {"brightness": 0.3,
                             "contrast": 0.3,
                             "saturation": 0.1,
                             'hue': 0.1})
        ],
        "p": 0.4,
    }),
    #("RandomGrayscale", {"p": 0.1}),
    ("ToTensor", {}),
    ('Normalize', {'mean': [0.485, 0.456, 0.406],
                   'std':[0.229, 0.224, 0.225]}),
]
#
config.augmentations_valid = [
    ("Resize", {'size': (config.img_size, config.img_size)}),
    ("ToTensor", {}),
    ('Normalize', {'mean': [0.485, 0.456, 0.406],
                   'std':[0.229, 0.224, 0.225]}),
]
#################
# train model
#################
config.backbone =  "ResNet-18"
config.projector_args = {
    'd_out': 768,
    'd_hidden': 768,
    'n_hidden': 2,
    'normalize': True,
    'dropout_rate': None,
    'activation_last': False,
    'normalize_last': False,
    'dropout_rate_last': None,
}
#################
# training
#################
config.batch_size = 256
config.num_epochs = 800
config.num_workers = 8

#################
# optimizer
#################
#config.optimizer = "sgd"
#config.optimizer_args = {
#        "lr": 1e-2,
#        "weight_decay": 1e-6,
#        "momentum": 0.9
#    }
config.optimizer = "adam"
config.optimizer_args = {
    'lr': 1e-3
}
config.scheduler = None #"cosine_decay"
config.scheduler_args = {
        "T_max": config.num_epochs,
        "eta_min": 0,
}
#################
# down train
#################
config.down_batch_size = 512
config.down_num_epochs = 1
config.down_num_workers = 8

#################
# down optimizer
#################
config.down_optimizer = "sgd"
config.down_optimizer_args = {
        "lr": 0.03 * config.down_batch_size / 256,
        "weight_decay": 5e-4,  # used always
        "momentum": 0.9
    }
config.down_scheduler = "cosine_decay"
config.down_scheduler_args = {
        "T_max": config.down_num_epochs,
        "eta_min": 0,
}

config.loss = {
    'lmda_rec': 100,
    'lmda_kld': 0.5
}
config.debug = False
config.p_base = "/mnt/experiments/siamesevae"
add_paths_to_confg(config)
config = DottedDict(config)

In [ ]:
# META VARS
P_CKPT = None
CONTINUE = True

In [ ]:
if P_CKPT is not None:
    print("LOADING CHECKPOINT {}".format(P_CKPT))
    ckpt = torch.load(P_CKPT)
    
    if CONTINUE:
        print("USING CKPT Config")
        config = ckpt["config"]

In [ ]:
class Decoder(nn.Module):
    def __init__(self, d_z, d_out, fc_hidden1=768, fc_hidden2=768):
        super(Decoder, self).__init__()
        #
        self.d_out = d_out
        self.d_z = d_z
        self.fc_hidden1=fc_hidden1,
        self.fc_hidden2=fc_hidden1
        #
        self.ch1, self.ch2, self.ch3, self.ch4 = 16, 32, 64, 128
        self.k1, self.k2, self.k3, self.k4 = (3, 3), (3, 3), (3, 3), (3, 3)      # 2d kernal size

         # Sampling vector
        self.fc4 = nn.Linear(self.d_z, self.fc_hidden2)
        self.fc_bn4 = nn.BatchNorm1d(self.fc_hidden2)
        self.fc5 = nn.Linear(self.fc_hidden2, 64 * 4 * 4)
        self.fc_bn5 = nn.BatchNorm1d(64 * 4 * 4)
        self.relu = nn.ReLU(inplace=True)
        
        # Decoder
        self.convTrans6 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3, stride=2,
                               padding=0),
            nn.BatchNorm2d(32, momentum=0.01),
            nn.ReLU(inplace=True),
        )
        self.convTrans7 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=3, stride=2,
                               padding=0),
            nn.BatchNorm2d(16, momentum=0.01),
            nn.ReLU(inplace=True),
        )

        self.convTrans8 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=16, out_channels=8, kernel_size=3, stride=2,
                               padding=0),
            nn.BatchNorm2d(8, momentum=0.01),
            nn.ReLU(inplace=True),
        )

        self.convTrans9 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=8, out_channels=3, kernel_size=3, stride=2,
                               padding=0),
            #nn.BatchNorm2d(3, momentum=0.01),
            nn.Sigmoid()    # y = (y1, y2, y3) \in [0 ,1]^3
        )

    def forward(self, x):
        x = self.relu(self.fc_bn4(self.fc4(x)))
        x = self.relu(self.fc_bn5(self.fc5(x))).view(-1, 64, 4, 4)
        x = self.convTrans6(x)
        x = self.convTrans7(x)
        x = self.convTrans8(x)
        x = self.convTrans9(x)
        x = F.interpolate(x, size=(self.d_out, self.d_out), mode='bilinear', align_corners=False)
        return x
    
    def forward_no_resize(self, x):
        x = self.relu(self.fc_bn4(self.fc4(x)))
        x = self.relu(self.fc_bn5(self.fc5(x))).view(-1, 64, 4, 4)
        x = self.convTrans6(x)
        x = self.convTrans7(x)
        x = self.convTrans8(x)
        x = self.convTrans9(x)
        return x

class ResnetVAE(torch.nn.Module):
    def __init__(self, backbone, net_means, net_logvars, decoder):
        super(ResnetVAE, self).__init__()

        self.decoder = decoder
        self.backbone = backbone
        self.net_means = net_means
        self.net_logvars = net_logvars
        
        self.dim_out = net_means[-1].out_features

    def encode(self, x):
        z = self.backbone(x)
        z_mu = self.net_means(z)

        # we predict log_var = log(std**2)
        # -> std = exp(0.5 * log_var)
        # -> alternative is to directly predict std ;)
        z_logvar = self.net_logvars(z)

        return z_mu, z_logvar
    
    def decode(self, x):
        return self.decoder(x)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return z

    def reparametrize(self, mu, logvar):
        #
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(logvar)
        #
        return eps * std + mu
        #std = logvar.mul(0.5).exp_()
        #eps = torch.autograd.Variable(std.data.new(std.size()).normal_())
        #eps = torch.randn_like(logvar)
        #return eps.mul(std).add_(mu)
        
    def loss_kld(self, mu, logvar):
        loss = 1 + logvar - mu ** 2 - logvar.exp()
        loss = -0.5 * torch.sum(loss, dim = 1)
        return loss

    def loss_rec(self, x1, x2):
        #loss = F.mse_loss(x1, x2)
        loss = F.mse_loss(x1, x2, reduction='mean')
        #loss = F.binary_cross_entropy(x1, x2, reduction='sum')
        #loss = ((x1 - x2)**2).mean(axis=1)
        #loss = - F.cosine_similarity(x1, x2)
        #p = F.normalize(x1, p=2, dim=1)
        #z = F.normalize(x2, p=2, dim=1)
        #loss = -(p * z).sum(dim=1)
        return loss

In [ ]:
# load data
dl_train, dl_down_train, dl_down_valid = get_dataloaders_from_config(config)
#
# create model
backbone = get_backbone(config.backbone, pretrained=False)

# projectors
projector_means = get_projector(d_in=backbone.dim_out, **config.projector_args)
projector_logvars = get_projector(d_in=backbone.dim_out, **config.projector_args)

#
decoder = Decoder(d_z=projector_means[-1].out_features, d_out=config.img_size)

#
model = ResnetVAE(backbone, projector_means, projector_logvars, decoder)

# optimizer
optimizer = get_optimizer(config.optimizer, model, config.optimizer_args)
if config.scheduler is not None:
    scheduler = get_scheduler(config.scheduler, optimizer, config.scheduler_args)
else:
    scheduler = None

In [ ]:
global_step = 0
epoch = 0
#
if P_CKPT is not None:
    r = model.load_state_dict(ckpt['model_state_dict'])
    print("Load model state dict", r)
    if CONTINUE:
        print("LOAD optimizer")
        optimizer.load_state_dict(ckpt['optimizer_state_dict'])
        #
        print("LOAD scheduler")
        scheduler.load_state_dict(ckpt['lr_scheduler_state_dict'])
        #
        global_step = ckpt['global_step']
        epoch = ckpt['global_epoch']
        print("Continue epoch {}, step {}".format(epoch, global_step))

In [ ]:
# tensorboard
writer = SummaryWriter(config.p_logs)

# create train dir
config.p_logs.mkdir(exist_ok=True, parents=True)
config.p_ckpts.mkdir(exist_ok=True, parents=True)
#
print("tensorboard --logdir={}".format(config.p_logs))

In [ ]:
model

## DEBUGGGING

In [ ]:
model = model.to(config.device)

In [ ]:
def dl_generator():
    for (x1, x2), target in dl_train:
        yield x1, x2
        
generator = dl_generator()

In [ ]:
x1, x2 = next(generator)
model.train()
x1, x2 = x1.to(config.device), x2.to(config.device)
optimizer.zero_grad()
#
mu1, logvar1 = model.encode(x1)
z1 = model.reparametrize(mu1, logvar1)
#
mu2, logvar2 = model.encode(x2)
z2 = model.reparametrize(mu2, logvar2)
#
x1_rec = model.decode(z1)
x2_rec = model.decode(z2)
#
loss_rec1 = model.loss_rec(x1_rec, x1)
loss_rec2 = model.loss_rec(x2_rec, x2)
#
loss_rec = (loss_rec1 + loss_rec2) / 2
#
## kld loss
loss_kld1 = model.loss_kld(mu1, logvar1).mean()
loss_kld2 = model.loss_kld(mu2, logvar2).mean()

loss_kld = (loss_kld1 + loss_kld2) / 2
    
loss = (config.loss.lmda_kld * loss_kld) + (config.loss.lmda_rec * loss_rec)
        
loss.backward()
optimizer.step()

## END DEBUGGING

In [ ]:

#
model = model.to(config.device)
for epoch in range(epoch, config.num_epochs, 1):
    # STD EVAL
    if epoch % config.freqs.std_eval == 0:
        std, cov = std_cov_valid(dl_down_valid, model, config.device)
        plt.matshow(cov)
        plt.colorbar()
        print("min {:.3f} max: {:.3f}".format(cov.min(), cov.max()))
        plt.show()
        #
        writer.add_scalar('std', std, global_step)

    # STD EVAL BACKBONE
    if epoch % config.freqs.std_eval == 0:
        std, cov = std_cov_valid(dl_down_valid, model.backbone, config.device)
        plt.matshow(cov)
        plt.colorbar()
        print("backbone min {:.3f} max: {:.3f}".format(cov.min(), cov.max()))
        plt.show()
        #
        writer.add_scalar('std_backbone', std, global_step)
        
    # KNN EVAL
    if epoch % config.freqs.knn_eval == 0:
        acc = down_knn(dl_down_valid, model, config.device, n_neighbors=5)
        #
        writer.add_scalar('acc_knn', acc, global_step)
    
        # KNN EVAL
    if epoch % config.freqs.knn_eval == 0:
        acc = down_knn(dl_down_valid, model.backbone, config.device, n_neighbors=5)
        #
        writer.add_scalar('acc_knn_back', acc, global_step)
    
    # LINEAR EVAL
    if epoch % config.freqs.lin_eval == 0:
        classifier = torch.nn.Linear(model.dim_out, config.n_classes).to(config.device)
        classifier.weight.data.normal_(mean=0.0, std=0.01)
        classifier.bias.data.zero_()
        #
        criterion = torch.nn.CrossEntropyLoss().to(config.device)
        #

        optimizer_down = get_optimizer(config.down_optimizer, classifier, config.down_optimizer_args)
        scheduler_down = get_scheduler(config.down_scheduler, optimizer_down, config.down_scheduler_args)
        #
        _, _ = down_train_linear(model, classifier, dl_down_train,
                              optimizer_down, config.device, config.down_num_epochs)
            
        acc = down_valid_linear(
                model,
                classifier,
                dl_down_valid,
                config.device)
        writer.add_scalar('acc_linear', acc, global_step)
    
    # REC
    if epoch % config.freqs.plot_rec == 0:
        model.eval()
        x_show, _ = next(iter(dl_down_valid))
        x_show = x_show[:10]
        with torch.no_grad():
            z = model(x_show.to(config.device))
            x_rec = model.decode(z)
        show_batch(x_show)
        show_batch(x_rec.detach().cpu())
    
    # TRAIN STEP
    losses, step = 0., 0.
    losses_kld = 0
    losses_rec = 0
    p_bar = tqdm(dl_train, desc=f'Pretrain {epoch}')
    for (x1, x2), target in p_bar:
        model.train()
        x1, x2 = x1.to(config.device), x2.to(config.device)
        optimizer.zero_grad()
        #
        mu1, logvar1 = model.encode(x1)
        z1 = model.reparametrize(mu1, logvar1)
        #
        mu2, logvar2 = model.encode(x2)
        z2 = model.reparametrize(mu2, logvar2)
        #
        x1_rec = model.decode(z1)
        x2_rec = model.decode(z2)
        #
        loss_rec1 = model.loss_rec(x1_rec, x1)
        loss_rec2 = model.loss_rec(x2_rec, x2)
        #
        loss_rec = (loss_rec1 + loss_rec2) / 2
        #
        ## kld loss
        loss_kld1 = model.loss_kld(mu1, logvar1).mean()
        loss_kld2 = model.loss_kld(mu2, logvar2).mean()

        loss_kld = (loss_kld1 + loss_kld2) / 2

        #loss = loss_rec
        #loss = (config.loss.lmda_kld * loss_kld) + (config.loss.lmda_rec * loss_rec)
        loss = loss_rec
        #loss = loss_rec
        
        loss.backward()
        optimizer.step()

        
        losses += loss.item()
        losses_kld += loss_kld.item()
        losses_rec += loss_rec.item()
        global_step += 1
        step += 1
        p_bar.set_postfix({'loss': losses / step, 'rec': losses_rec / step, 'kld': losses_kld / step})
        #
        writer.add_scalar('loss', loss.item(), global_step)
        writer.add_scalar('rec loss', loss_rec.item(), global_step)
        writer.add_scalar('kld loss', loss_kld.item(), global_step)
        writer.add_scalar('kld1 loss', loss_kld1.item(), global_step)
        writer.add_scalar('kld2 loss', loss_kld2.item(), global_step)
        #
        writer.add_scalar('rec1 loss', loss_rec1.item(), global_step)
        writer.add_scalar('rec2 loss', loss_rec2.item(), global_step)

        
    writer.add_scalar('epoch loss', losses / step, global_step)
    
    # CHECKPOINTING
    if epoch % config.freqs.ckpt == 0 and epoch != 0:
        p_ckpt = config.p_ckpts / config.fs_ckpt.format(config.dataset, epoch)
        config.p_ckpts.mkdir(exist_ok=True, parents=True)
        #
        save_checkpoint(model, optimizer, scheduler, config, epoch, global_step, p_ckpt)
        print('\nSave model for epoch {} at {}'.format(epoch, p_ckpt))
    writer.add_scalar('epoch', epoch, global_step)